In [11]:
import pandas as pd
import openpyxl
import streamlit as st
import geopandas as gpd
import folium
from streamlit_folium import st_folium
import json

2025-05-05 11:12:51.977 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [6]:
populacao_df = pd.read_csv('data/populacao.csv', encoding='utf-8', sep=';')
populacao_df = populacao_df.sort_values(by='Municipio', ascending=True)
populacao_df.reset_index(drop=True, inplace=True)
populacao_df.to_csv('populacao.csv', index=False, encoding='utf-8')


In [12]:
data = openpyxl.load_workbook(
    'data/Anexo 3 - Solicitação Quantidade Serviços Prestados por Tipo BA GERAL - '
    'Atualizado 20250409.xlsx', 
    data_only=True
)

data_frota = openpyxl.load_workbook(
    'data/Novo Anexo 1 - Solicitação de Frota BA 2022_2023_2024_2025 em 20250107.xlsx', 
    data_only=True
)

In [9]:
clinicas_cols = [
    'Id_Município Cidadão', 
    'Número CIRETRAN Município Cidadão', 
    'Nome CIRETRAN Município Cidadão', 
    'Município Cidadão',
    'Id_Município Clínica', 
    'Número CIRETRAN Clínica', 
    'Nome CIRETRAN Clínica', 
    'Município Clínica', 
    'CNPJ', 
    'Razão Social', 
    'Exames Médicos', 
    'Exames Psicológicos', 
    'Total', 
    'Percentual'
]

cfc_cols = [
    'Id_Município Cidadão', 
    'Número CIRETRAN Município Cidadão', 
    'Nome CIRETRAN Município Cidadão', 
    'Município Cidadão',
    'Id_Município CFC', 
    'Número CIRETRAN CFC', 
    'Nome CIRETRAN CFC', 
    'Município CFC', 
    'CNPJ', 
    'Razão Social', 
    'Cursos Teóricos',
    'Cursos Práticos', 
    'Total', 
    'Percentual'
]

epiv_cols = [
    'Id_Município', 
    'Número CIRETRAN', 
    'Nome CIRETRAN', 
    'Município',
    'CNPJ', 
    'Razão Social', 
    'Estampagem', 
    'Total', 
    'Percentual'
]

patio_cols = [
    'Id_Município', 
    'Número CIRETRAN', 
    'Nome CIRETRAN', 
    'Município',
    'CNPJ', 
    'Razão Social', 
    'Veículos removidos', 
    'Total', 
    'Percentual'
]

ecv_cols = [
    'Id_Município',
    'Número CIRETRAN',
    'Nome CIRETRAN',
    'Município',
    'CNPJ',
    'Razão Social',
    'Vistoria Lacrada Veículo 4 Rodas Até 16 Lugares ou Maior 3,5T',
    'Vistoria Lacrada Veículo Carga PBT Mais 3,5T',
    'Vistoria Lacrada Veículo Combinado Veículo P/Unidade',
    'Vistoria Lacrada Veículo Passageiro com Lotação Acima de 16 Lugares',
    'Vistoria Lacrada Veículo 2 ou 3 Rodas',
    'Vistoria RENAVE de Veículo 4 Rodas 16 Lugares ou Até 3,5 Ton',
    'Vistoria RENAVE de Veículos de 2 e 3 Rodas',
    'Vistoria Veículo Carga com PBT Acima de 3,5T',
    'Vistoria Veicular de Combinações de Veículos por Unidade',
    'Vistoria Veículo 2 ou 3 Rodas',
    'Vistoria Veículo 4 Rodas Até 16 Lugares ou Até 3,5 Ton',
    'Vistoria Veículo Passageiros com Capacidade Acima de 16 Lugares',
    'Outras',
    'Total',
    'Percentual'
]

frota_cols = [
    'Id_Município', 
    'Número CIRETRAN', 
    'Nome CIRETRAN', 
    'Município',
    'Automóvel',
    'Caminhão',
    'Caminhonete',
    'Microonibus',
    'Moto',
    'Motor-Casa',
    'Onibus',
    'Reboque',
    'Trator',
    'Outros',
    'Total', 
    'Percentual'
]

In [13]:
frota_24 = data_frota['2025']
frota_df_24 = pd.DataFrame(frota_24.values)
frota_df_24 = frota_df_24.drop(columns=[16,17,18,19,20,21,22,23,24,25,26])
frota_df_24.columns = frota_cols
frota_df_24 = frota_df_24.drop([0, 1, 2, 3])
frota_df_24 = frota_df_24.iloc[:-2]
frota_df_24 = frota_df_24.reset_index(drop=True)
frota_df_24['Percentual'] = frota_df_24['Percentual'].apply(lambda x: round(x * 100, 2))
frota_df_24

,Id_Município,Número CIRETRAN,Nome CIRETRAN,Município,Automóvel,Caminhão,Caminhonete,Microonibus,Moto,Motor-Casa,Onibus,Reboque,Trator,Outros,Total,Percentual
0,,0,SEDE,SALVADOR,719197,27537,141704,4620,213446,152,12072,17607,101,9,1136445,20.59
1,3871,1,CIR. SANTO AMARO,SANTO AMARO,5370,339,1236,180,6221,1,255,301,0,2,13905,0.25
2,3037,1,CIR. SANTO AMARO,SAUBARA,657,41,189,51,807,0,4,50,0,4,1803,0.03
3,3927,1,CIR. SANTO AMARO,TEODORO SAMPAIO,497,33,118,21,403,0,14,22,0,0,1108,0.02
4,3933,1,CIR. SANTO AMARO,TERRA NOVA,909,47,142,22,669,0,26,44,0,0,1859,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,3495,33,CIR. CRUZ DAS ALMAS,CRUZ DAS ALMAS,13706,1476,3323,420,18315,3,155,953,0,0,38351,0.69
413,3283,33,CIR. CRUZ DAS ALMAS,ITATIM,2071,427,672,61,2452,1,49,265,0,0,5998,0.11
414,3869,33,CIR. CRUZ DAS ALMAS,SANTA TERESINHA,1029,50,208,21,584,0,49,44,0,0,1985,0.04
415,3881,33,CIR. CRUZ DAS ALMAS,SAO FELIPE,2331,372,866,65,3977,0,78,91,0,0,7780,0.14


In [16]:
frota_df_24['Id_Município'][frota_df_24['Município'] == 'SALVADOR'] = 3849
frota_df_24

C:\Users\caiop\AppData\Local\Temp\ipykernel_26780\2117343639.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  frota_df_24['Id_Município'][frota_df_24['Município'] == 'SALVADOR'] = 3849
C:\Users\caiop\AppData\Local\Temp\ipykernel_26780\211

,Id_Município,Número CIRETRAN,Nome CIRETRAN,Município,Automóvel,Caminhão,Caminhonete,Microonibus,Moto,Motor-Casa,Onibus,Reboque,Trator,Outros,Total,Percentual
0,3849,0,SEDE,SALVADOR,719197,27537,141704,4620,213446,152,12072,17607,101,9,1136445,20.59
1,3871,1,CIR. SANTO AMARO,SANTO AMARO,5370,339,1236,180,6221,1,255,301,0,2,13905,0.25
2,3037,1,CIR. SANTO AMARO,SAUBARA,657,41,189,51,807,0,4,50,0,4,1803,0.03
3,3927,1,CIR. SANTO AMARO,TEODORO SAMPAIO,497,33,118,21,403,0,14,22,0,0,1108,0.02
4,3933,1,CIR. SANTO AMARO,TERRA NOVA,909,47,142,22,669,0,26,44,0,0,1859,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,3495,33,CIR. CRUZ DAS ALMAS,CRUZ DAS ALMAS,13706,1476,3323,420,18315,3,155,953,0,0,38351,0.69
413,3283,33,CIR. CRUZ DAS ALMAS,ITATIM,2071,427,672,61,2452,1,49,265,0,0,5998,0.11
414,3869,33,CIR. CRUZ DAS ALMAS,SANTA TERESINHA,1029,50,208,21,584,0,49,44,0,0,1985,0.04
415,3881,33,CIR. CRUZ DAS ALMAS,SAO FELIPE,2331,372,866,65,3977,0,78,91,0,0,7780,0.14


In [5]:
from unidecode import unidecode

texto = "Brejolândia"
texto_sem_acento = unidecode(texto) 
 # resultado: "Brejolandia"

texto_sem_acento


'Brejolandia'

In [3]:
import pandas as pd

In [4]:
df_populacao = pd.read_csv('populacao.csv')
df_populacao

,Id_Municipio,Municipio,Populacao
0,3301,ABAIRA,7452.0
1,3303,ABARE,18240.0
2,3305,ACAJUTIBA,14215.0
3,3253,ADUSTINA,14554.0
4,3307,AGUA FRIA,14871.0
...,...,...,...
412,3965,VITORIA DA CONQUISTA,394024.0
413,3967,WAGNER,9847.0
414,3999,WANDERLEY,13436.0
415,3969,WENCESLAU GUIMARAES,25430.0


In [7]:
municipios_pop = {unidecode(nome): str(id_mun) 
                 for nome, id_mun in zip(df_populacao['Municipio'], df_populacao['Id_Municipio'])}
municipios_pop

{'ABAIRA': '3301',
 'ABARE': '3303',
 'ACAJUTIBA': '3305',
 'ADUSTINA': '3253',
 'AGUA FRIA': '3307',
 'AIQUARA': '3311',
 'ALAGOINHAS': '3313',
 'ALCOBACA': '3315',
 'ALMADINA': '3317',
 'AMARGOSA': '3319',
 'AMELIA RODRIGUES': '3321',
 'AMERICA DOURADA': '3071',
 'ANAGE': '3323',
 'ANDARAI': '3325',
 'ANDORINHA': '3255',
 'ANGICAL': '3327',
 'ANGUERA': '3329',
 'ANTAS': '3331',
 'ANTONIO CARDOSO': '3333',
 'ANTONIO GONCALVES': '3335',
 'APORA': '3337',
 'APUAREMA': '3257',
 'ARACAS': '3259',
 'ARACATU': '3339',
 'ARACI': '3341',
 'ARAMARI': '3343',
 'ARATACA': '3073',
 'ARATUIPE': '3345',
 'AURELINO LEAL': '3347',
 'BAIANOPOLIS': '3349',
 'BAIXA GRANDE': '3351',
 'BANZAE': '3261',
 'BARRA': '3353',
 'BARRA DA ESTIVA': '3355',
 'BARRA DO CHOCA': '3357',
 'BARRA DO MENDES': '3359',
 'BARRA DO ROCHA': '3361',
 'BARREIRAS': '3363',
 'BARRO ALTO': '3075',
 'BARRO PRETO': '3365',
 'BARROCAS': '3077',
 'BELMONTE': '3367',
 'BELO CAMPO': '3369',
 'BIRITINGA': '3371',
 'BOA NOVA': '3373',
 'B